# Building a Spam Filter with Naive Bayes

In this project, we're going to study the practical side of the algorithm by building a spam filter for SMS messages.

To classify messages as spam or non-spam, the computer:

1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages — probabilities for spam and non-spam.
3. Classifies a new message based on these probability values — if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

So our first task is to "teach" the computer how to classify messages. To do that, we'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). We can also download the dataset directly [from this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where we can also find some of the authors' papers.

## Exploring the Dataset

Let's start by reading in the dataset.

In [1]:
import pandas as pd

dataset = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

dataset.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
print(dataset.shape)

dataset['Label'].value_counts(normalize=True)*100

(5572, 2)


ham     86.593683
spam    13.406317
Name: Label, dtype: float64

## Training and Test Set

We read in the dataset and saw that about 87% of the messages are ham ("ham" means non-spam), and the remaining 13% are spam. Now that we've become a bit familiar with the dataset, we can move on to building the spam filter.

However, before creating it, it's very helpful to first think of a way of testing how well it works. When creating software (a spam filter is software), a good rule of thumb is that designing the test comes before creating the software. If we write the software first, then it's tempting to come up with a biased test just to make sure the software passes it.

Once our spam filter is done, we'll need to test how good it is with classifying *new* messages. To test the spam filter, we're first going to split our dataset into two categories:

+ A **training set**, which we'll use to "train" the computer how to classify messages.
+ A **test set**, which we'll use to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

+ The training set will have 4,458 messages (about 80% of the dataset).
+ The test set will have 1,114 messages (about 20% of the dataset).

To better understand the purpose of putting a test set aside, let's begin by observing that all 1,114 messages in our test set are already classified by a human. When the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Once we have the results, we'll be able to compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

For this project, our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

Let's create a training and a test set. We're going to start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset.

In [3]:
randomized_set = dataset.sample(frac=1, random_state=1)

training_set = randomized_set[:4458].reset_index(drop=True)
test_set = randomized_set[4458:].reset_index(drop=True)

print(training_set.shape)
training_set['Label'].value_counts(normalize=True)*100

(4458, 2)


ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [4]:
print(test_set.shape)
test_set['Label'].value_counts(normalize=True)*100

(1114, 2)


ham     86.804309
spam    13.195691
Name: Label, dtype: float64

## Letter Case and Punctuation

We split our dataset into a training set and a test set. The next big step is to use the training set to teach the algorithm to classify new messages.

When a new message comes in, our Naive Bayes algorithm will make the classification based on the results it gets to these two equations:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Let's also summarize what the terms in the equations above mean:

\begin{aligned}
&N_{w_i|Spam} = \text{the number of times the word } w_i \text{ occurs in spam messages} \\
&N_{w_i|Spam^C} = \text{the number of times the word } w_i \text{ occurs in non-spam messages} \\
\\
&N_{Spam} = \text{total number of words in spam messages} \\
&N_{Spam^C} = \text{total number of words in non-spam messages} \\
\\
&N_{Vocabulary} = \text{total number of words in the vocabulary} \\
&\alpha = 1 \ \ \ \ (\alpha \text{ is a smoothing parameter})
\end{aligned}

To calculate all these probabilities, we first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need. Right now, our training and test sets have this format (the messages are fictitious to make the example easier to understand):

![](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_1.png)

To make the calculations easier, we want bring the data to this format (the table below is a transformation of the table you see above):

![](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_2.png)

About the transformation above, notice that:

+ The `SMS` column doesn't exist anymore.
+ Instead, the `SMS` column is replaced by a series of new columns, where each column represents a unique word from the vocabulary.
+ Each row describes a single message. For instance, the first row corresponds to the message "SECRET PRIZE! CLAIM SECRET PRIZE NOW!!", and it has the values `spam, 2, 2, 1, 1, 0, 0, 0, 0, 0`. These values tell us that:
  + The message is spam.
  + The word "secret" occurs two times inside the message.
  + The word "prize" occurs two times inside the message.
  + The word "claim" occurs one time inside the message.
  + The word "now" occurs one time inside the message.
  + The words "coming", "to", "my", "party", and "winner" occur zero times inside the message.
+ All words in the vocabulary are in lower case, so "SECRET" and "secret" come to be considered to be the same word.
+ Punctuation is not taken into account anymore (for instance, we can't look at the table and conclude that the first message initially had three exclamation marks).

Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower case.

In [5]:
# Before cleaning
training_set.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [6]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()

# After cleaning
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


We removed the punctuation and changed all letters to lowercase.

## Creating the Vocabulary

With the exception of the "Label" column, every other column in the transformed table above represents a unique word in our vocabulary (more specifically, each column shows the frequency of that unique word for any given message). Recall from the previous mission that we call the set of unique words a **vocabulary**.

Let's create a list with all of the unique words that occur in the messages of our training set.

In [7]:
# Splitting each word on the SMS column
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []

for words in training_set['SMS']:
    for word in words:
        vocabulary.append(word)
        
# Transform the vocabulary list into a set to remove duplicates
vocabulary = set(vocabulary)

# Transform the vocabulary set back into a list
vocabulary = list(vocabulary)

len(vocabulary) # Unique words

7783

We managed to create the vocabulary for our messages in the training set. Now we're going to use the vocabulary to make the data transformation we need.

## The Final Training Set

Eventually, we're going to create a new DataFrame. However, we'll first build a dictionary that we'll then convert to the DataFrame we need.

To create the dictionary we need for our training set, we can use the code below, where:

+ We start by initializing a dictionary named `word_counts_per_sms`, where each key is a unique word (a string) from the vocabulary, and each value is a list of the length of training set, where each element in the list is a `0`.
  + The code `[0] * 5` outputs `[0, 0, 0, 0, 0]`. So the code `[0] * len(training_set['SMS'])` outputs a list of the length of `training_set['SMS']`, where each element in the list will be a `0`.
+ We loop over `training_set['SMS']` using at the same time the `enumerate()` function to get both the index and the SMS message (`index` and `sms`).
  + Using a nested loop, we loop over `sms` (where `sms` is a list of strings, where each string represents a word in a message).
    + We incremenent `word_counts_per_sms[word][index]` by `1`.
    
Now that we have the dictionary we need, let's do the final transformations to our training set and then move forward with creating the spam filter.

In [8]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,duo,spose,weirdest,grumble,assessment,loosu,eastenders,2bold,6times,fuckinnice,...,paying,logos,readiness,owns,offense,speak,vomiting,suzy,nose,take
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
final_training_set = pd.concat([training_set, word_counts], axis=1)

final_training_set.head()

,Label,SMS,duo,spose,weirdest,grumble,assessment,loosu,eastenders,2bold,...,paying,logos,readiness,owns,offense,speak,vomiting,suzy,nose,take
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter.

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, recall that we need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Some of the terms in the four equations above will have the same value for every new message. As a start, let's first calculate:

+ P(Spam) and P(Ham)
+ N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

Recall that:

+ N<sub>Spam</sub> is equal to the number of words in all the spam messages — it's *not* equal to the number of spam messages, and it's not equal to the total number of *unique* words in spam messages.
+ N<sub>Ham</sub> is equal to the number of words in all the non-spam messages — it's *not* equal to the number of non-spam messages, and it's not equal to the total number of *unique* words in non-spam messages.

We'll also use Laplace smoothing and set $\alpha = 1$

In [10]:
# Isolating spam and ham messages first
spam_messages = final_training_set[final_training_set['Label'] == 'spam']
ham_messages = final_training_set[final_training_set['Label'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(final_training_set)
p_ham = len(ham_messages) / len(final_training_set)

# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calculating Parameters

On the previous screen, we managed to calculate a few terms for our equations:

+ P(Spam) and P(Ham)
+ N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

As we've already mentioned, all these terms will have constant values in our equations for every new message (regardless of the message or each individual word in the message).

However, P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) will vary depending on the individual words. For instance, P("secret"|Spam) will have a certain probability value, while P("cousin"|Spam) or P("lovely"|Spam) will most likely have other values.

Although both P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) vary depending on the word, the probability for each individual word is constant for every new message.

For instance, let's say we receive two new messages:

+ "secret code"
+ "secret party 2night"

We'll need to calculate P("secret"|Spam) for both these messages, and we can use the training set to get the values we need to find a result for the equation below:

\begin{equation}
P(\text{"secret"}|Spam) = \frac{N_{"secret"|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

The steps we take to calculate P("secret"|Spam) will be identical for both of our new messages above, or for any other new message that contains the word "secret". The key detail here is that calculating P("secret"|Spam) only depends on the training set, and as long as we don't make changes to the training set, P("secret"|Spam) stays constant. The same reasoning also applies to P("secret"|Ham).

This means that we can use our training set to calculate the probability for each word in our vocabulary. If our vocabulary contained only the words "lost", "navigate", and "sea", then we'd need to calculate six probabilities:

+ P("lost"|Spam) and P("lost"|Ham)
+ P("navigate"|Spam) and P("navigate"|Ham)
+ P("sea"|Spam) and P("sea"|Ham)

We have 7,783 words in our vocabulary, which means we'll need to calculate a total of 15,566 probabilities. For each word, we need to calculate both P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham).

In more technical language, the probability values that P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) will take are called **parameters**.

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in. Imagine the algorithm will be used to classify 1,000,000 new messages. Why repeat all these calculations 1,000,000 times when we could just do them once at the beginning?

Let's now calculate all the parameters using the equations below:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

Recall that P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) are key parts of the equations that we need to classify the new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

In [11]:
# Initiate parameters
parameters_spam = {unique_word: 0 for unique_word in vocabulary}
parameters_ham = {unique_word: 0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()  
    p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + (alpha * n_vocabulary))
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    parameters_ham[word] = p_word_given_ham

## Classifying A New Message

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

+ Take in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>)
+ Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>)
+ Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
  + If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
  + If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
  + If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [12]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam   
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]   

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

Let's use the classify() function to classify two new messages: one message is obviously spam, and the other is obviously ham. We can use these two messages:

+ `'WINNER!! This is the secret code to unlock the money: C3421.'`
+ `"Sounds good, Tom, then see u there"`

In [13]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [14]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


## Measuring the Spam Filter's Accuracy

We managed to create a spam filter, and we classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,114 messages.

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Note that, in training, our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm.

First off, we'll change the classify() function that we wrote previously to return the labels instead of printing them.

In [15]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam   
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
        
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]   

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [16]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use **accuracy** as a metric:

\begin{equation}
\text{Accuracy} = \frac{\text{number of correctly classified messages}}{\text{total number of classified messages}}
\end{equation}

In [17]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
        
accuracy = correct / total
print(accuracy)

0.9874326750448833


## Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 98.74% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.